In [3]:



# Adapted from: github.com/aneesha/RAKE/rake.py
from __future__ import division
import operator
import nltk
import string

def isPunct(word):
    return len(word) == 1 and word in string.punctuation

def isNumeric(word):
    try:
        float(word) if '.' in word else int(word)
        return True
    except ValueError:
        return False

class RakeKeywordExtractor:

    def __init__(self):
        self.stopwords = set(nltk.corpus.stopwords.words('russian'))
        self.top_fraction = 1 #consider top third candidate keywords by score

    def _generate_candidate_keywords(self, sentences):
        phrase_list = []
        for sentence in sentences:
            words = map(lambda x: "|" if x in self.stopwords else x,
                nltk.word_tokenize(sentence.lower()))
            phrase = []
            for word in words:
                if word == "|" or isPunct(word):
                    if len(phrase) > 0:
                        phrase_list.append(phrase)
                        phrase = []
                else:
                    phrase.append(word)
        return phrase_list

    def _calculate_word_scores(self, phrase_list):
        word_freq = nltk.FreqDist()
        word_degree = nltk.FreqDist()
        for phrase in phrase_list:
            degree = len(list(filter(lambda x: not isNumeric(x), phrase))) - 1
            for word in phrase:
                word_freq[word] +=1
                word_degree[word] += degree
        for word in word_freq.keys():
            word_degree[word] = word_degree[word] + word_freq[word] # itself
        # word score = deg(w) / freq(w)
        word_scores = {}
        for word in word_freq.keys():
            word_scores[word] = float(word_degree[word]) / word_freq[word]
        return word_scores

    def _calculate_phrase_scores(self, phrase_list, word_scores):
        phrase_scores = {}
        for phrase in phrase_list:
            phrase_score = 0
            for word in phrase:
                phrase_score += word_scores[word]
                phrase_scores[" ".join(phrase)] = phrase_score
        return phrase_scores
    
    def extract(self, text, incl_scores=False):
        sentences = nltk.sent_tokenize(text)
        phrase_list = self._generate_candidate_keywords(sentences)
        word_scores = self._calculate_word_scores(phrase_list)
        phrase_scores = self._calculate_phrase_scores(
            phrase_list, word_scores)
        sorted_phrase_scores = sorted(phrase_scores.items(),
            key=operator.itemgetter(1), reverse=True)
        n_phrases = len(sorted_phrase_scores)
        if incl_scores:
            return sorted_phrase_scores[0:int(float(n_phrases)/self.top_fraction)]
        else:
            return map(lambda x: x[0],
                sorted_phrase_scores[0:int(float(n_phrases)/self.top_fraction)])

def test():
    rake = RakeKeywordExtractor()
    text = """
    Одной из популярных позиций нашего производства является угловой диван Респект. Если Вы современный и деловой человек, любите комфорт и уют, а каждый сантиметр жилплощади для Вас на вес золота – тогда угловой диван Респект отлично впишется в интерьер Вашей квартиры, заняв при этом минимум места. Угловой диван Респект не только практичен в использовании, удобен, мягок и современен, но и еще удивительно компактен. В его изготовлении используются только самые экологически чистые ткани и наполнители, а механизм раскладки еврокнижка - поможет Вам сэкономить массу усилий и времени для его преобразования в полноценную кровать.

     Модель со схемой раскладки еврокнижка, сделана на основе деревянного каркаса с использованием пружины типа Боннель в паре с пружинной змейкой.
Особенностью этого дивана является небольшой габаритный размер короткой части модели 135см, что позволяет устанавливать его в наши малогабаритные квартиры. Стильные боковые перила в виде полудуги придают угловому дивану Респект чрезвычайной элегантности.

     Как и в большинстве моделей нашего производства в модели Респект есть возможность добавления на спальную и сидячую части натуральных наполнителей (кокос, латекс, спрут, шерсть и др.), что придаст модели более выраженный ортопедический эффект. В конструкцию углового дивана заложено два больших места для хранения постельного белья, одно под сидячей продольной частью, второе в угловой части. Угловой диван Респект комплектуется тремя большими подушками для удобного сидения в собранном виде.

     Все материалы, которые используются при производстве углового дивана Респект проверены и сертифицированы, на весь модельный ряд и в частности на эту модель распространяется гарантийное и послегарантийное обслуживание.
    """
#     """Compatibility of systems of linear constraints over the set of natural 
#         numbers. Criteria of compatibility of a system of linear Diophantine 
#         equations, strict inequations, and nonstrict inequations are considered. 
#         Upper bounds for components of a minimal set of solutions and algorithms 
#         of construction of minimal generating sets of solutions for all types of 
#         systems are given. These criteria and the corresponding algorithms for 
#         constructing a minimal supporting set of solutions can be used in solving 
#         all the considered types of systems and systems of mixed types."""
    keywords = rake.extract(text, incl_scores=False)
    return keywords
  

In [9]:
rake = RakeKeywordExtractor()
text = """
Одной из популярных позиций нашего производства является угловой диван Респект. Если Вы современный и деловой человек, любите комфорт и уют, а каждый сантиметр жилплощади для Вас на вес золота – тогда угловой диван Респект отлично впишется в интерьер Вашей квартиры, заняв при этом минимум места. Угловой диван Респект не только практичен в использовании, удобен, мягок и современен, но и еще удивительно компактен. В его изготовлении используются только самые экологически чистые ткани и наполнители, а механизм раскладки еврокнижка - поможет Вам сэкономить массу усилий и времени для его преобразования в полноценную кровать.

 Модель со схемой раскладки еврокнижка, сделана на основе деревянного каркаса с использованием пружины типа Боннель в паре с пружинной змейкой.
Особенностью этого дивана является небольшой габаритный размер короткой части модели 135см, что позволяет устанавливать его в наши малогабаритные квартиры. Стильные боковые перила в виде полудуги придают угловому дивану Респект чрезвычайной элегантности.

 Как и в большинстве моделей нашего производства в модели Респект есть возможность добавления на спальную и сидячую части натуральных наполнителей (кокос, латекс, спрут, шерсть и др.), что придаст модели более выраженный ортопедический эффект. В конструкцию углового дивана заложено два больших места для хранения постельного белья, одно под сидячей продольной частью, второе в угловой части. Угловой диван Респект комплектуется тремя большими подушками для удобного сидения в собранном виде.

 Все материалы, которые используются при производстве углового дивана Респект проверены и сертифицированы, на весь модельный ряд и в частности на эту модель распространяется гарантийное и послегарантийное обслуживание.
    """
#     """Compatibility of systems of linear constraints over the set of natural 
#         numbers. Criteria of compatibility of a system of linear Diophantine 
#         equations, strict inequations, and nonstrict inequations are considered. 
#         Upper bounds for components of a minimal set of solutions and algorithms 
#         of construction of minimal generating sets of solutions for all types of 
#         systems are given. These criteria and the corresponding algorithms for 
#         constructing a minimal supporting set of solutions can be used in solving 
#         all the considered types of systems and systems of mixed types."""
keywords = rake.extract(text, incl_scores=True)

In [10]:
list(keywords)

[('дивана является небольшой габаритный размер короткой части модели 135см',
  68.83333333333334),
 ('виде полудуги придают угловому дивану респект чрезвычайной элегантности',
  58.42857142857143),
 ('популярных позиций нашего производства является угловой диван респект',
  52.67857142857143),
 ('угловой диван респект комплектуется тремя большими подушками',
  44.17857142857143),
 ('угловой диван респект отлично впишется', 26.17857142857143),
 ('производстве углового дивана респект проверены', 25.92857142857143),
 ('большинстве моделей нашего производства', 20.0),
 ('конструкцию углового дивана заложено', 18.5),
 ('сидячую части натуральных наполнителей', 17.0),
 ('угловой диван респект', 16.17857142857143),
 ('самые экологически чистые ткани', 16.0),
 ('использованием пружины типа боннель', 16.0),
 ('угловой части', 10.0),
 ('модели респект', 9.761904761904763),
 ('хранения постельного белья', 9.0),
 ('схемой раскладки еврокнижка', 9.0),
 ('выраженный ортопедический эффект', 9.0),
 ('

In [4]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_colwidth', 1000)
res = pd.DataFrame(test())
res

TypeError: data argument can't be an iterator

In [14]:
from rutermextract import TermExtractor

In [19]:
term_extractor = TermExtractor()
text = u"""Одной из популярных позиций нашего производства является угловой диван Респект. Если Вы современный и деловой человек, любите комфорт и уют, а каждый сантиметр жилплощади для Вас на вес золота – тогда угловой диван Респект отлично впишется в интерьер Вашей квартиры, заняв при этом минимум места. Угловой диван Респект не только практичен в использовании, удобен, мягок и современен, но и еще удивительно компактен. В его изготовлении используются только самые экологически чистые ткани и наполнители, а механизм раскладки еврокнижка - поможет Вам сэкономить массу усилий и времени для его преобразования в полноценную кровать.

     Модель со схемой раскладки еврокнижка, сделана на основе деревянного каркаса с использованием пружины типа Боннель в паре с пружинной змейкой.
Особенностью этого дивана является небольшой габаритный размер короткой части модели 135см, что позволяет устанавливать его в наши малогабаритные квартиры. Стильные боковые перила в виде полудуги придают угловому дивану Респект чрезвычайной элегантности.

     Как и в большинстве моделей нашего производства в модели Респект есть возможность добавления на спальную и сидячую части натуральных наполнителей (кокос, латекс, спрут, шерсть и др.), что придаст модели более выраженный ортопедический эффект. В конструкцию углового дивана заложено два больших места для хранения постельного белья, одно под сидячей продольной частью, второе в угловой части. Угловой диван Респект комплектуется тремя большими подушками для удобного сидения в собранном виде.

     Все материалы, которые используются при производстве углового дивана Респект проверены и сертифицированы, на весь модельный ряд и в частности на эту модель распространяется гарантийное и послегарантийное обслуживание."""
for term in term_extractor(text):
    print(term.normalized, term.count)

респект 7
диван 5
угловой 4
модель 4
угловой диван 3
наше производство 2
еврокнижка 2
три большими подушками 1
стильные боковые перила 1
сидячая продольная часть 1
небольшая габаритный размер 1
наши малогабаритные квартира 1
короткая часть модели 1
каждый сантиметр жилплощади 1
использование пружины типа 1
два больших места 1
выраженный ортопедический эффект 1
весь модельный ряд 1
чрезвычайная элегантность 1
чистые ткань 1
удобный сидение 1
угловой части 1
схема раскладки 1
собранный вид 1
сидячая часть 1
пружинная змейка 1
постельный бельё 1
послегарантийное обслуживание 1
популярные позиции 1
полноценная кровать 1
натуральные наполнители 1
минимум места 1
механизм раскладки 1
масса усилий 1
деревянный каркас 1
деловая человек 1
возможность добавления 1
вид полудуги 1
вес золота 1
ваша квартира 1
большинство моделей 1
шерсть 1
частность 1
хранение 1
уют 1
спрут 1
спальная 1
производство 1
преобразование 1
пар 1
особенность 1
основа 1
наполнители 1
материалы 1
латекс 1
конструкция 1
ко

In [41]:
term_extracted = term_extractor(text)

In [51]:
term.word_count

1